### Libraries

#### Boto3:
Python SDK for AWS. It allows you to directly create, update, and delete AWS resources from your Python scripts

In [1]:
import boto3

In [2]:
import pandas as pd

#### io:
The io module in Python provides facilities for working with streams of data in memory or on disk. The StringIO class in the io module is used for creating a stream object that behaves like a file object, but is backed by a string buffer in memory instead of a physical file on disk.

In [3]:
from io import StringIO

In [4]:
import time

### Access Case

In [5]:
AWS_ACCESS_KEY = 'your_access_key'
AWS_SECRET_KEY = 'your secret key'
AWS_REGION = 'us-east-1'
SCHEMA_NAME = 'infectious-diseases-nepal-db' # Database name
S3_STAGING_DIR = 's3://infectious-diseases-nepal-athena-output/output/'
S3_BUCKET_NAME = 'infectious-diseases-nepal-athena-output'
S3_OUTPUT_DIRECTORY = 'output'

### Connect to Athena and Query Data

In [6]:
athena_client = boto3.client(
                "athena",
                aws_access_key_id=AWS_ACCESS_KEY,
                aws_secret_access_key=AWS_SECRET_KEY,
                region_name=AWS_REGION
)

### Function
This function basically take boto3 object and dictionary and run query in Athena and store the output in S3 i.e in staging_dir

In [7]:
Dict = {}
def download_and_load_query_results(client: boto3.client, query_response: Dict) -> pd.DataFrame:
    while True:
        try:
            #This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id = AWS_ACCESS_KEY,
        aws_secret_access_key = AWS_SECRET_KEY,
        region_name = AWS_REGION
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)

### Query Response

In [8]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM infectious_diseases_nepal",
    QueryExecutionContext = {"Database":SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    }
)

In [9]:
response

{'QueryExecutionId': '6e4bd615-a8f2-4267-b591-cd5e7bc2e8b9',
 'ResponseMetadata': {'RequestId': '9f1ddfe3-c0af-4106-91b2-e62cb846ecef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 May 2023 16:00:22 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9f1ddfe3-c0af-4106-91b2-e62cb846ecef'},
  'RetryAttempts': 0}}

In [10]:
nepal_infectious_diseases = download_and_load_query_results(athena_client, response)

In [11]:
nepal_infectious_diseases.head()

,gho (code),gho (display),gho (url),publishstate (code),publishstate (display),publishstate (url),year (code),year (display),year (url),region (code),...,country (code),country (display),country (url),display value,numeric,low,high,stderr,stddev,comments
0,#indicator+code,#indicator+name,#indicator+url,#status+code,#status+name,NaN,NaN,NaN,NaN,#region+code,...,#country+code,#country+name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1965.0,1965.0,NaN,SEAR,...,NPL,Nepal,NaN,97.0,97.0,NaN,NaN,NaN,NaN,NaN
2,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1993.0,1993.0,NaN,SEAR,...,NPL,Nepal,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1994.0,1994.0,NaN,SEAR,...,NPL,Nepal,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1995.0,1995.0,NaN,SEAR,...,NPL,Nepal,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


### Transformation: 

#### Slicing Data Frame

In [12]:
nepal_infectious_diseases = nepal_infectious_diseases[1:] # take all the data except 0 index

In [13]:
nepal_infectious_diseases.head()

,gho (code),gho (display),gho (url),publishstate (code),publishstate (display),publishstate (url),year (code),year (display),year (url),region (code),...,country (code),country (display),country (url),display value,numeric,low,high,stderr,stddev,comments
1,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1965.0,1965.0,NaN,SEAR,...,NPL,Nepal,NaN,97.0,97.0,NaN,NaN,NaN,NaN,NaN
2,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1993.0,1993.0,NaN,SEAR,...,NPL,Nepal,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1994.0,1994.0,NaN,SEAR,...,NPL,Nepal,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1995.0,1995.0,NaN,SEAR,...,NPL,Nepal,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
5,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,PUBLISHED,Published,NaN,1971.0,1971.0,NaN,SEAR,...,NPL,Nepal,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN


#### Checking equality of two columns:

In [14]:
are_equal = nepal_infectious_diseases['startyear'] == nepal_infectious_diseases['endyear']

# Display the result
if not are_equal.all():
    print(nepal_infectious_diseases[~are_equal]) # Note that the ~ operator is used to negate the boolean series, and the .all() method checks if all values in the series are True.

#### Removing redundant columns

In [15]:
columns_to_remove = ['publishstate (code)', 'publishstate (display)', 'publishstate (url)', 'year (display)', 'year (url)', 
                     'region (code)', 'region (url)', 'worldbankincomegroup (code)', 'startyear', 'endyear', 
                     'worldbankincomegroup (display)', 'worldbankincomegroup (url)', 'country (code)', 'country (url)', 'numeric',
                    'low', 'high', 'stderr', 'stddev', 'comments']

In [16]:
nepal_infectious_diseases = nepal_infectious_diseases.drop(columns_to_remove, axis=1)

In [17]:
nepal_infectious_diseases.head()

,gho (code),gho (display),gho (url),year (code),region (display),country (display),display value
1,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1965.0,South-East Asia,Nepal,97.0
2,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1993.0,South-East Asia,Nepal,0.0
3,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1994.0,South-East Asia,Nepal,0.0
4,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1995.0,South-East Asia,Nepal,0.0
5,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1971.0,South-East Asia,Nepal,0.0


#### Removing column whose display value is 0

In [18]:
nepal_infectious_diseases = nepal_infectious_diseases[nepal_infectious_diseases['display value'] != 0]

In [19]:
nepal_infectious_diseases.head()

,gho (code),gho (display),gho (url),year (code),region (display),country (display),display value
1,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1965.0,South-East Asia,Nepal,97.0
10,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1976.0,South-East Asia,Nepal,1.0
11,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1977.0,South-East Asia,Nepal,4.0
12,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1978.0,South-East Asia,Nepal,10.0
15,CHOLERA_0000000002,Number of reported deaths from cholera,https://www.who.int/data/gho/indicator-metadat...,1989.0,South-East Asia,Nepal,2.0


### Storing latest Data Frame into S3 bucket

In [20]:
bucket = "infectious-diseases-nepal-analytics"

In [21]:
csv_buffer = StringIO()

In [22]:
csv_buffer

In [23]:
nepal_infectious_diseases.to_csv(csv_buffer, index=False)

In [24]:
csv_buffer.seek(0)

0

In [25]:
s3_resource = boto3.resource('s3',
                              aws_access_key_id=AWS_ACCESS_KEY,
                              aws_secret_access_key= AWS_SECRET_KEY)
s3_resource.Object(bucket, 'nepal_infectious_diseases_analytics.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'RNGMFYE8XAY9XTD1',
  'HostId': 'YbaWsTEoDjqHO9ve4rQACM55JhuwGhXpDu+o4AtVZUS9hHQ+7NQX8pqYbuX8PAidgi+FXcllaxg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'YbaWsTEoDjqHO9ve4rQACM55JhuwGhXpDu+o4AtVZUS9hHQ+7NQX8pqYbuX8PAidgi+FXcllaxg=',
   'x-amz-request-id': 'RNGMFYE8XAY9XTD1',
   'date': 'Thu, 11 May 2023 16:04:10 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"9b3f8f606199aa97018ab2a50d141705"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"9b3f8f606199aa97018ab2a50d141705"',
 'ServerSideEncryption': 'AES256'}